In [ ]:
!pip install bs4
!pip install pygooglenews
!pip install requests
!pip install datetime
!pip install pandas
!pip install numpy
!pip install urllib3
!pip install vaderSentiment
!pip install scipy
!pip install vaderSentiment
!pip install statsmodels
!pip install regex==2022.3.2

In [ ]:
import pandas as pd
import numpy as np
from urllib.request import urlretrieve
import datetime
import requests
from pygooglenews import GoogleNews
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import io

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from google.colab import  drive
drive.mount('/drive')

In [ ]:
from pygooglenews import GoogleNews
gn = GoogleNews()
 
ticker_df = pd.read_csv(io.BytesIO(uploaded['index1.csv']))
ticker_df['Name'] = ticker_df['Name'].astype(str)
BSE_SENSEX_tickers = list(ticker_df['Name'])
BSE_SENSEX_weights = list(ticker_df['Weight'])

print(BSE_SENSEX_tickers)

NSE_NIFTY_tickers = [] #

In [ ]:
search = gn.search('COVID')
newsitem = search['entries']
newsitem[0]

In [ ]:
def webpage_text_fetch(url):

    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
    }
    
    try:
        html = requests.get(url, headers=headers, timeout=10).text
        print('request successful')
    except Exception:
        text = "no text"
        return text
    except requests.exceptions.ConnectionError:
        text = "no text"
        return text
    except TypeError:
        text = "no text"
        return text
    except requests.exceptions.HTTPError:
        text = "no text"
        return text
    except requests.exceptions.Timeout:
        text = "no text"
        return text
    except requests.exceptions.SSLError:
        text = "no text"
        return text
    except requests.exceptions.TooManyRedirects:
        text = "no text"
        return text
        
    
    soup = BeautifulSoup(html, features="html.parser")
    print('bs4 successful')
    for script in soup(["script", "style"]):
        script.extract()    
    
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    
    return text

In [ ]:
def story_gen(ticker):
    stories = []
    start_date = datetime.datetime(2020, 12, 30)
    date_format = '%a, %d %b %Y %H:%M:%S %Z'
    while(start_date <= datetime.datetime(2021, 6, 30)):
        end_date = start_date + datetime.timedelta(days=15)
        search = gn.search(ticker, from_ = start_date.strftime('%Y-%m-%d'), to_ = end_date.strftime('%Y-%m-%d'))
        newsitem = search['entries']
        for item in newsitem:
            story = {
                'title':item.title,
                'date':datetime.datetime.strptime(item.published, date_format),
                'link':item.link,
            }
            stories.append(story)
        start_date += datetime.timedelta(days=15)
    
    return stories

In [ ]:
news_story_df_list = []
story_list_list = []
news_story_df_out_test = []
story_title_df_out_list = []
story_date_df_out_list = []
story_link_df_out_list = []

check = 0

for ticker in BSE_SENSEX_tickers:
    
    print(ticker)
    stories = story_gen(ticker)
        
    story_list_list.append(stories)
    text_list = []
    news_story_df_test = []
    story_title_df_list = []
    story_date_df_list = []
    story_link_df_list = []
    
    for i in range(len(stories)):
      story_title_df_list.append(stories[i]['title'])
      story_date_df_list.append(stories[i]['date'])
      story_link_df_list.append(stories[i]['link'])
      print('Fetch Attempt')
      text = webpage_text_fetch(stories[i]['link'])
      
      print('Fetch Successfull: ', check)
      check += 1
      text_list.append(text)
      news_story_df_test.append(text)
      print('Append Successful')

    
    print("No. of Stories assessed: ", len(stories))
    temp1_df = pd.DataFrame(text_list, columns =[ticker])
    temp2_df = pd.DataFrame(story_title_df_list, columns =[ticker])
    temp3_df = pd.DataFrame(story_date_df_list, columns =[ticker])
    temp4_df = pd.DataFrame(story_link_df_list, columns =[ticker])
    temp5_df = pd.DataFrame(news_story_df_test, columns =[ticker])

    news_story_df_list.append(temp1_df)
    story_title_df_out_list.append(temp2_df)
    story_date_df_out_list.append(temp3_df)
    story_link_df_out_list.append(temp4_df)
    news_story_df_out_test.append(temp5_df)

    
    news_story_test_df = pd.concat(news_story_df_out_test, axis=1)
    story_title_df = pd.concat(story_title_df_out_list, axis=1)
    story_date_df = pd.concat(story_date_df_out_list, axis=1)
    story_link_df = pd.concat(story_link_df_out_list, axis=1)
    news_story_test_df = pd.concat(news_story_df_out_test, axis=1)

    #news_story_test_df.to_csv('/drive/My Drive/SOP/news_df.csv',mode='w+')
    story_title_df.to_csv('/drive/My Drive/SOP/title_df.csv',mode='w+')
    story_date_df.to_csv('/drive/My Drive/SOP/date_df.csv',mode='w+')
    story_link_df.to_csv('/drive/My Drive/SOP/link_df.csv',mode='w+')
    

In [ ]:
print('News text data fetch Complete')

In [ ]:
news_story_df = pd.concat(news_story_df_list, axis=1).copy()
from google.colab import files
type(news_story_df['RELIANCE'][0])
for tickers in BSE_SENSEX_tickers:
  news_story_df[ticker] = news_story_df[ticker].astype(str)
  for i in range(len(news_story_df[ticker])):
    news_story_df[ticker][i] = news_story_df[ticker][i].encode('utf-8','ignore').decode("utf-8")
  
#news_story_df.to_csv('news_df.csv') 
#files.download('news_df.csv')

In [ ]:
analyzer = SentimentIntensityAnalyzer()
total_story_sum = 0
index = 0
list_df = []
list_df2 = []
list_df3 = []

for ticker in BSE_SENSEX_tickers:
    
    print(ticker)
    stories = story_list_list[index]
    index += 1
    total_story_sum += len(stories)
    
    ticker_story_list = []
    ticker_pos_list = []
    ticker_neg_list = []
    
    ticker_score = ticker + '_score' 
    ticker_pos = ticker + '_pos'
    ticker_neg = ticker + '_neg'
    
    check = 0
    for i in range(len(stories)):
        check = 1
                
        text = news_story_df[ticker].iloc[i]        
        title_analyzed = analyzer.polarity_scores((stories[i])['title'])
        text_analyzed = analyzer.polarity_scores(text)
        
        score = ((title_analyzed['compound']) + (text_analyzed['compound']))/2
        pos = ((title_analyzed['pos']) + (text_analyzed['pos']))/2
        neg = ((title_analyzed['neg']) + (text_analyzed['neg']))/2
        
        ticker_story_dict = {
            'date': (stories[i])['date'],
            ticker_score : score
        }
        ticker_story_list.append(ticker_story_dict)
        
        ticker_pos_dict = {
            'date': (stories[i])['date'],
            ticker_pos : pos
        }
        ticker_pos_list.append(ticker_pos_dict)
        
        ticker_neg_dict = {
            'date': (stories[i])['date'],
            ticker_neg : neg
        }
        ticker_neg_list.append(ticker_neg_dict)
        
    if(check==0):
        ticker_story = {
            'date': datetime.datetime(2021, 1, 1),
            ticker_score : 0
        }
        ticker_story_list.append(ticker_story)
        
        ticker_pos = {
            'date': datetime.datetime(2021, 1, 1),
            ticker_pos : 0
        }
        ticker_pos_list.append(ticker_pos)
        
        ticker_neg = {
            'date': datetime.datetime(2021, 1, 1),
            ticker_neg : 0
        }
        ticker_neg_list.append(ticker_neg)
        
        
    df = pd.DataFrame(ticker_story_list)
    df = df.groupby(['date']).mean()
    list_df.append(df)
    
    df2 = pd.DataFrame(ticker_pos_list)
    df2 = df2.groupby(['date']).mean()
    list_df2.append(df2)
    
    df3 = pd.DataFrame(ticker_neg_list)
    df3 = df3.groupby(['date']).mean()
    list_df3.append(df3)


In [ ]:
date_col = []
date_var = datetime.datetime(2020, 12, 30)
while(date_var != datetime.datetime(2021, 6, 30)):
    date_col.append(date_var)
    date_var += datetime.timedelta(days=1)
    
date_column_df = pd.DataFrame(date_col)
date_column_df.columns = ['date'] 
date_column_df['date'] = pd.to_datetime(date_column_df['date']).dt.date

In [ ]:
list_df_copy = list_df.copy()
merged_df = pd.concat(list_df_copy, axis=1)
merged_df = merged_df.reset_index()
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.date
final_merged_df = pd.merge(date_column_df, merged_df, on='date',how='left')
BSE_SENSEX_weights = list(ticker_df['Weight'])
cols = list(final_merged_df.columns[1:])
final_merged_df = final_merged_df.fillna(0)
final_merged_df['Index_Score'] = np.average(final_merged_df[cols], weights=BSE_SENSEX_weights , axis=1)
final_merged_df_score = final_merged_df
final_merged_df

In [ ]:
list_df_copy = list_df2.copy()
merged_df = pd.concat(list_df_copy, axis=1)
merged_df = merged_df.reset_index()
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.date
final_merged_df = pd.merge(date_column_df, merged_df, on='date',how='left')
BSE_SENSEX_weights = list(ticker_df['Weight'])
cols = list(final_merged_df.columns[1:])
final_merged_df = final_merged_df.fillna(0)
final_merged_df['Index_Pos'] = np.average(final_merged_df[cols], weights=BSE_SENSEX_weights , axis=1)
final_merged_df_pos = final_merged_df
final_merged_df_pos

In [ ]:
list_df_copy = list_df3.copy()
merged_df = pd.concat(list_df_copy, axis=1)
merged_df = merged_df.reset_index()
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.date
final_merged_df = pd.merge(date_column_df, merged_df, on='date',how='left')
BSE_SENSEX_weights = list(ticker_df['Weight'])
cols = list(final_merged_df.columns[1:])
final_merged_df = final_merged_df.fillna(0)
final_merged_df['Index_Neg'] = np.average(final_merged_df[cols], weights=BSE_SENSEX_weights , axis=1)
final_merged_df_neg = final_merged_df
final_merged_df_neg

In [ ]:
main_df = final_merged_df_score[['date','Index_Score']]
main_df2 = final_merged_df_pos[['date','Index_Pos']]
main_df3 = final_merged_df_neg[['date','Index_Neg']]
main_df

In [ ]:
stock_df = pd.read_csv(io.BytesIO(uploaded['index1data.csv']))
stock_df

In [ ]:
stock_df['date'] = pd.to_datetime(stock_df['Date']).dt.date

In [ ]:
stock_df = stock_df.drop(['Date'], axis=1)
stock_df 

In [ ]:
stock_df['DR'] = np.log((stock_df['Close']/ stock_df['Close'].shift(1)))
stock_df['DV7'] = ''
stock_df['DV3'] = ''

for i in range(7):
    col_name = 'shift'+str(i)
    stock_df[col_name] = stock_df['Close'].shift(i)


In [ ]:
for i in range(stock_df.shape[0]):
    stock_df['DV7'][i] = np.sqrt(np.var([stock_df[('shift'+str(j))][i] for j in range(7)]))
    stock_df['DV3'][i] = np.sqrt(np.var([stock_df[('shift'+str(j))][i] for j in range(3)]))
    
stock_df['DV7-1'] = stock_df['DV7'].shift(-1)
stock_df['DV3-1'] = stock_df['DV3'].shift(-1)

In [ ]:
stock_df

In [ ]:
main_merged_df = pd.merge(stock_df, main_df, on='date',how='inner')
main_merged_df = main_merged_df[['Close','DR','DV3','DV7','DV3-1','DV7-1','Index_Score']]
main_merged_df['DV7'] = pd.to_numeric(main_merged_df['DV7'])
main_merged_df['DV7-1'] = pd.to_numeric(main_merged_df['DV7-1'])
main_merged_df['DV3'] = pd.to_numeric(main_merged_df['DV3'])
main_merged_df['DV3-1'] = pd.to_numeric(main_merged_df['DV3-1'])

main_merged_df2 = pd.merge(stock_df, main_df2, on='date',how='inner')
main_merged_df2 = main_merged_df2[['Close','DR','DV3','DV7','DV3-1','DV7-1','Index_Pos']]
main_merged_df2['DV7'] = pd.to_numeric(main_merged_df2['DV7'])
main_merged_df2['DV7-1'] = pd.to_numeric(main_merged_df2['DV7-1'])
main_merged_df2['DV3'] = pd.to_numeric(main_merged_df2['DV3'])
main_merged_df2['DV3-1'] = pd.to_numeric(main_merged_df2['DV3-1'])

main_merged_df3 = pd.merge(stock_df, main_df3, on='date',how='inner')
main_merged_df3 = main_merged_df3[['Close','DR','DV3','DV7','DV3-1','DV7-1','Index_Neg']]
main_merged_df3['DV7'] = pd.to_numeric(main_merged_df3['DV7'])
main_merged_df3['DV7-1'] = pd.to_numeric(main_merged_df3['DV7-1'])
main_merged_df3['DV3'] = pd.to_numeric(main_merged_df3['DV3'])
main_merged_df3['DV3-1'] = pd.to_numeric(main_merged_df3['DV3-1'])

In [ ]:
main_df

In [ ]:
import numpy as np
from scipy.stats import pearsonr

print("Index_Score, Price:", pearsonr(main_merged_df['Index_Score'],main_merged_df['Close']))
print("Index_Score, DR:", pearsonr(main_merged_df['Index_Score'],main_merged_df['DR']))
print("Index_Score, DV7:", pearsonr(main_merged_df['Index_Score'],main_merged_df['DV7']))
print("Index_Score, DV7-1:", pearsonr(main_merged_df['Index_Score'],main_merged_df['DV7-1']))
print("Index_Score, DV3:", pearsonr(main_merged_df['Index_Score'],main_merged_df['DV3']))
print("Index_Score, DV3-1:", pearsonr(main_merged_df['Index_Score'],main_merged_df['DV3-1']))

In [ ]:
print("Index_Pos, Price:", pearsonr(main_merged_df2['Index_Pos'],main_merged_df2['Close']))
print("Index_Pos, DR:", pearsonr(main_merged_df2['Index_Pos'],main_merged_df2['DR']))
print("Index_Pos, DV7:", pearsonr(main_merged_df2['Index_Pos'],main_merged_df2['DV7']))
print("Index_Pos, DV7-1:", pearsonr(main_merged_df2['Index_Pos'],main_merged_df2['DV7-1']))
print("Index_Pos, DV3:", pearsonr(main_merged_df2['Index_Pos'],main_merged_df2['DV3']))
print("Index_Pos, DV3-1:", pearsonr(main_merged_df2['Index_Pos'],main_merged_df2['DV3-1']))

In [ ]:
print("Index_Neg, Price:", pearsonr(main_merged_df3['Index_Neg'],main_merged_df3['Close']))
print("Index_Neg, DR:", pearsonr(main_merged_df3['Index_Neg'],main_merged_df3['DR']))
print("Index_Neg, DV7:", pearsonr(main_merged_df3['Index_Neg'],main_merged_df3['DV7']))
print("Index_Neg, DV7-1:", pearsonr(main_merged_df3['Index_Neg'],main_merged_df3['DV7-1']))
print("Index_Neg, DV3:", pearsonr(main_merged_df3['Index_Neg'],main_merged_df3['DV3']))
print("Index_Neg, DV3-1:", pearsonr(main_merged_df3['Index_Neg'],main_merged_df3['DV3-1']))

In [ ]:
from statsmodels.tsa.stattools import adfuller

print("ADF Close", adfuller(main_merged_df['Close']))
print("ADF DR", adfuller(main_merged_df['DR']))
print("ADF DV7", adfuller(main_merged_df['DV7']))
print("ADF DV7-1", adfuller(main_merged_df['DV7-1']))
print("ADF DV3", adfuller(main_merged_df['DV3']))
print("ADF DV3-1", adfuller(main_merged_df['DV3-1']))
print("ADF Index_Score", adfuller(main_merged_df['Index_Score']))
print("ADF Index_Pos", adfuller(main_merged_df2['Index_Pos']))
print("ADF Index_Neg", adfuller(main_merged_df3['Index_Neg']))

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

print("IS - DV7", grangercausalitytests(main_merged_df[['Index_Score', 'DV7']], maxlag=[1]))
print()
print("DV7 - IS", grangercausalitytests(main_merged_df[['DV7','Index_Score']], maxlag=[1]))
print()
print("IS - DV7-1", grangercausalitytests(main_merged_df[['Index_Score', 'DV7-1']], maxlag=[1]))
print()
print("DV7 - IS-1", grangercausalitytests(main_merged_df[['DV7-1','Index_Score']], maxlag=[1]))
print()
print("IS - DV3", grangercausalitytests(main_merged_df[['Index_Score', 'DV3']], maxlag=[1]))
print()
print("DV3 - IS", grangercausalitytests(main_merged_df[['DV3','Index_Score']], maxlag=[1]))
print()
print("IS - DV3-1", grangercausalitytests(main_merged_df[['Index_Score', 'DV3-1']], maxlag=[1]))
print()
print("DV3 - IS-1", grangercausalitytests(main_merged_df[['DV3-1','Index_Score']], maxlag=[1]))
print()

In [ ]:
print("IS - DV7", grangercausalitytests(main_merged_df2[['Index_Pos', 'DV7']], maxlag=[1]))
print()
print("DV7 - IS", grangercausalitytests(main_merged_df2[['DV7','Index_Pos']], maxlag=[1]))
print()
print("IS - DV7-1", grangercausalitytests(main_merged_df2[['Index_Pos', 'DV7-1']], maxlag=[1]))
print()
print("DV7 - IS-1", grangercausalitytests(main_merged_df2[['DV7-1','Index_Pos']], maxlag=[1]))
print()
print("IS - DV3", grangercausalitytests(main_merged_df2[['Index_Pos', 'DV3']], maxlag=[1]))
print()
print("DV3 - IS", grangercausalitytests(main_merged_df2[['DV3','Index_Pos']], maxlag=[1]))
print()
print("IS - DV3-1", grangercausalitytests(main_merged_df2[['Index_Pos', 'DV3-1']], maxlag=[1]))
print()
print("DV3 - IS-1", grangercausalitytests(main_merged_df2[['DV3-1','Index_Pos']], maxlag=[1]))
print()

In [ ]:
print("IS - DV7", grangercausalitytests(main_merged_df3[['Index_Neg', 'DV7']], maxlag=[1]))
print()
print("DV7 - IS", grangercausalitytests(main_merged_df3[['DV7','Index_Neg']], maxlag=[1]))
print()
print("IS - DV7-1", grangercausalitytests(main_merged_df3[['Index_Neg', 'DV7-1']], maxlag=[1]))
print()
print("DV7 - IS-1", grangercausalitytests(main_merged_df3[['DV7-1','Index_Neg']], maxlag=[1]))
print()
print("IS - DV3", grangercausalitytests(main_merged_df3[['Index_Neg', 'DV3']], maxlag=[1]))
print()
print("DV3 - IS", grangercausalitytests(main_merged_df3[['DV3','Index_Neg']], maxlag=[1]))
print()
print("IS - DV3-1", grangercausalitytests(main_merged_df3[['Index_Neg', 'DV3-1']], maxlag=[1]))
print()
print("DV3 - IS-1", grangercausalitytests(main_merged_df3[['DV3-1','Index_Neg']], maxlag=[1]))
print()